In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ResearchProject/datafiles/001/Dexcom_001.csv')

In [ ]:
df

,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer)
0,1,NaN,FirstName,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,LastName,NaN,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,PatientIdentifier,NaN,2019-001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,DateOfBirth,NaN,1990-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,Device,NaN,NaN,Dexcom G6 Mobile App,iPhone G6,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,2569,2020-02-22 17:33:23,EGV,NaN,NaN,NaN,iPhone G6,139.0,NaN,NaN,NaN,NaN,789294.0
2569,2570,2020-02-22 17:38:23,EGV,NaN,NaN,NaN,iPhone G6,141.0,NaN,NaN,NaN,NaN,789594.0
2570,2571,2020-02-22 17:43:23,EGV,NaN,NaN,NaN,iPhone G6,143.0,NaN,NaN,NaN,NaN,789894.0
2571,2572,2020-02-22 17:48:22,EGV,NaN,NaN,NaN,iPhone G6,139.0,NaN,NaN,NaN,NaN,790194.0


In [ ]:
def glucoseGrabber (data):
  start_index = data["Timestamp (YYYY-MM-DDThh:mm:ss)"].first_valid_index()
  relevent = data.iloc[start_index:]
  return relevent[["Timestamp (YYYY-MM-DDThh:mm:ss)", "Glucose Value (mg/dL)"]].rename(columns={"Timestamp (YYYY-MM-DDThh:mm:ss)": "datetime", "Glucose Value (mg/dL)" : "glucose"})

def glucoseInterpolater(data):
  glucose = glucoseGrabber(data)
  del data
  glucose.loc[:, "datetime"] = pd.to_datetime(glucose["datetime"])
  glucose.set_index("datetime", inplace=True)
  ds_glucose = glucose.resample("5min").mean()
  return ds_glucose

In [ ]:
df = glucoseInterpolater(df)
df

,glucose
datetime,
2020-02-13 17:20:00,61.0
2020-02-13 17:25:00,59.0
2020-02-13 17:30:00,58.0
2020-02-13 17:35:00,59.0
2020-02-13 17:40:00,63.0
...,...
2020-02-22 17:30:00,139.0
2020-02-22 17:35:00,141.0
2020-02-22 17:40:00,143.0


In [ ]:
df['interval'] = (df['glucose'].notna() != df['glucose'].notna().shift()).cumsum()

# Create new dataframes for each interval
interval_dfs = {}
for interval_id, interval_group in df.groupby('interval'):
    if interval_group['glucose'].notna().any():
        interval_dfs[interval_id] = interval_group.drop(columns='interval')

# Access individual dataframes using interval IDs
for interval_id, interval_df in interval_dfs.items():
    print(f"Interval {interval_id}:")
    print(interval_df)4
    print("\n")

len(interval_dfs)

Interval 1:
                     glucose
datetime                    
2020-02-13 17:20:00     61.0
2020-02-13 17:25:00     59.0
2020-02-13 17:30:00     58.0
2020-02-13 17:35:00     59.0
2020-02-13 17:40:00     63.0
...                      ...
2020-02-15 08:40:00     62.0
2020-02-15 08:45:00     65.0
2020-02-15 08:50:00     69.0
2020-02-15 08:55:00     74.0
2020-02-15 09:00:00     77.0

[477 rows x 1 columns]


Interval 3:
                     glucose
datetime                    
2020-02-15 09:10:00     83.0
2020-02-15 09:15:00     86.0
2020-02-15 09:20:00     88.0
2020-02-15 09:25:00     91.0
2020-02-15 09:30:00     95.0
...                      ...
2020-02-19 23:15:00    111.0
2020-02-19 23:20:00    112.0
2020-02-19 23:25:00    112.0
2020-02-19 23:30:00    112.0
2020-02-19 23:35:00    112.0

[1326 rows x 1 columns]


Interval 5:
                     glucose
datetime                    
2020-02-20 02:45:00    112.0
2020-02-20 02:50:00    111.0
2020-02-20 02:55:00    111.0
2020-02-20 0

3

{1:                      glucose
 datetime                    
 2020-02-13 17:20:00     61.0
 2020-02-13 17:25:00     59.0
 2020-02-13 17:30:00     58.0
 2020-02-13 17:35:00     59.0
 2020-02-13 17:40:00     63.0
 ...                      ...
 2020-02-15 08:40:00     62.0
 2020-02-15 08:45:00     65.0
 2020-02-15 08:50:00     69.0
 2020-02-15 08:55:00     74.0
 2020-02-15 09:00:00     77.0
 
 [477 rows x 1 columns],
 3:                      glucose
 datetime                    
 2020-02-15 09:10:00     83.0
 2020-02-15 09:15:00     86.0
 2020-02-15 09:20:00     88.0
 2020-02-15 09:25:00     91.0
 2020-02-15 09:30:00     95.0
 ...                      ...
 2020-02-19 23:15:00    111.0
 2020-02-19 23:20:00    112.0
 2020-02-19 23:25:00    112.0
 2020-02-19 23:30:00    112.0
 2020-02-19 23:35:00    112.0
 
 [1326 rows x 1 columns],
 5:                      glucose
 datetime                    
 2020-02-20 02:45:00    112.0
 2020-02-20 02:50:00    111.0
 2020-02-20 02:55:00    111.0
 2020-